# Visualisering af DMI Vejrdata i en GUI

## Formål

Du skal hente aktuelle eller historiske vejrdata fra DMI’s API, indlæse dem i en pandas DataFrame, og præsentere resultaterne i en PySide6-applikation. Applikationen skal bruge Matplotlib som backend til at vise grafer eller bruge plotly til en interaktivt dashboard.

## Trin 1: Hent data fra DMI API

- DMI stiller åbne data til rådighed via DMI’s [Open Data](https://opendatadocs.dmi.govcloud.dk/en/Getting_Started) API.
- For at få adgang, skal man oprette en konto og generere en gratis API-nøgle på [DMI API Portal](https://opendatadocs.dmi.govcloud.dk/en/Authentication)
- Lav et Python-script, der henter data (JSON-format) og gemmer det i en pandas DataFrame. [Her](https://opendatadocs.dmi.govcloud.dk/en/APIs/Meteorological_Observation_API) finder du eksempler for **Meteorological Observation API**
    - Tip: brug [**requests**](https://pypi.org/project/requests/) til at hente data.
    - Husk at konvertere tidspunkter til **datetime**.

In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
# Din personlige API-nøgle fra DMI
with open("api_key_dmi.txt") as f:
    API_KEY = f.readline()

# Endpoint til observationer
BASE_URL = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"

# Parameter-eksempel:
# stationId: 06186 = København (Landbohøjskolen)
# datetime: seneste 6 timer
station_id = "06186"
hours = 6
now = datetime.now(timezone.utc)
start = now - timedelta(hours=hours)
datetime_range = f"{start.isoformat()}/{now.isoformat()}"
    # -> "2025-11-03T09:00:00+00:00/2025-11-03T15:00:00+00:00"

params = {
        "stationId": station_id,
        "datetime": datetime_range,
        "parameterId": "temp_mean_past1h",
        "api-key": API_KEY
    }

response = requests.get(BASE_URL, params=params)
if response.status_code != 200:
    raise RuntimeError(f"API fejl {response.status_code}: {response.text}")

data = response.json()
records = []
for item in data["features"]:
    props = item["properties"]
    records.append({
        "timeObserved": pd.to_datetime(props["observed"]),
        "value": props["value"]
        })

df = pd.DataFrame(records)
df.sort_values("timeObserved", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,timeObserved,value
0,2025-11-03 17:00:00+00:00,10.2
1,2025-11-03 18:00:00+00:00,10.1
2,2025-11-03 19:00:00+00:00,9.8
3,2025-11-03 20:00:00+00:00,9.7
4,2025-11-03 21:00:00+00:00,9.9


## Trin 2: Vælg et datasæt (fx temperatur over de seneste dage).

Udregn fx:

- gennemsnitstemperatur pr. dag
- højeste og laveste temperatur

## Trin 3: Lav en PySide6 GUI med Matplotlib

Byg en simpel GUI med PySide6. GUI’en skal have:
 
- En knap til at hente/loade data fra DMI.
- En figur (Matplotlib-plot) til at vise data.


## Ekstra udfordringer:
- Brug en DateTimePicker i PySide6, så man kan vælge tidsperiode.
- En dropdown-menu, hvor man kan vælge parameter (fx temperatur, nedbør).
- Tilføj mulighed for at vælge forskellige stationer (fx København, Aalborg).
- Lav et interaktivt `plotly.express` line plot af temperatur over tid.